In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with BigQuery DataFrames

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/bq_dataframes_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/bq_dataframes_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/bq_dataframes_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10

## Overview

BigQuery DataFrames (also known as BigFrames) provides a Pythonic DataFrame and machine learning (ML) API powered by the BigQuery engine.

* `bigframes.pandas` provides a pandas-like API for analytics.
* `bigframes.ml` provides a scikit-learn-like API for ML.
* `bigframes.ml.llm` provides API for large language models including Gemini.

You can learn more about [BigQuery DataFrames](https://cloud.google.com/bigquery/docs/bigquery-dataframes-introduction) and its [API reference](https://cloud.google.com/python/docs/reference/bigframes/latest).

For any issues or feedback please reach out to bigframes-feedback@google.com.

## Before you begin

Complete the tasks in this section to set up your environment.

### Install the python package

You need the [bigframes](https://pypi.org/project/bigframes/) python package to be installed. If you don't have that, uncomment and run the following cell and *restart the kernel*.

In [ ]:
#%pip install  --upgrade

### Set your project id and location

Following are some quick references:

* Google Cloud Project: https://cloud.google.com/resource-manager/docs/creating-managing-projects.
* BigQuery Location: https://cloud.google.com/bigquery/docs/locations.

In [ ]:
PROJECT_ID = ""  # @param {type: "string"}
LOCATION = "US"  # @param {type: "string"}

### Import library

In [ ]:
import bigframes.pandas as bpd


### Set BigQuery DataFrames options

In [ ]:
# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bpd.options.bigquery.project = PROJECT_ID

# Note: The location option is not required.
# It defaults to the location of the first table or query
# passed to read_gbq(). For APIs where a location can't be
# auto-detected, the location defaults to the "US" location.
bpd.options.bigquery.location = LOCATION

# Note: BigQuery DataFrames objects are by default fully ordered like Pandas.
# If ordering is not important for you, you can uncomment the following
# expression to run BigQuery DataFrames in partial ordering mode.
#bpd.options.bigquery.ordering_mode = "partial"

If you want to reset the project and/or location of the created DataFrame or Series objects, reset the session by executing `bpd.close_session()`. After that, you can redo the above steps.

# Query using the BigQuery magics

Running this code will query a table in BigQuery create a DataFrame named `_bq_df`.
Learn more in the [Visualizing BigQuery data guide in notebooks](https://cloud.google.com/bigquery/docs/visualize-jupyter).

This example uses a
[penguin public dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ml_datasets&t=penguins&page=table&_ga=2.251359750.1031997792.1692116300-1119797950.1692116300).


In [ ]:
%%bqsql
SELECT * FROM `bigquery-public-data.ml_datasets.penguins`

## Create a BigQuery DataFrames DataFrame

You can create a BigQuery DataFrames DataFrame by reading data from any of the
following locations:

* A local data file
* Data stored in a BigQuery table
* A data file stored in Cloud Storage
* An in-memory pandas DataFrame

Note that the DataFrame does not copy the data to the local memory, instead
keeps the underlying data in a BigQuery table during read and analysis. That's
how it can handle really large size of data (at BigQuery Scale) independent of
the local memory.

For simplicity, speed and cost efficiency, this tutorial uses the
[`penguins`](https://pantheon.corp.google.com/bigquery?ws=!1m5!1m4!4m3!1sbigquery-public-data!2sml_datasets!3spenguins)
table from BigQuery public data, which contains 27 KB data about a set of
penguins - species, island of residence, culmen length and depth, flipper length
and sex. There is a version of this data in the Cloud Storage
[cloud samples data](https://pantheon.corp.google.com/storage/browser/_details/cloud-samples-data/vertex-ai/bigframe/penguins.csv)
as well.

In [ ]:
# This is how you read a BigQuery table
df = bpd.read_gbq("bigquery-public-data.ml_datasets.penguins")

# This is how you would read a csv from the Cloud Storage
#df = bpd.read_csv("gs://cloud-samples-data/vertex-ai/bigframe/penguins.csv")

We can use `peek` to preview a few rows (selected arbitrarily) from the dataframes:

In [ ]:
df.peek()

We just created a DataFrame, `df`, refering to the entirety of the source table data, without downloading it to the local machine.

## Inspect and manipulate data in BigQuery DataFrames

### Using pandas API

You can use pandas API on the BigQuery DataFrames DataFrame as you normally would in Pandas, but computation happens in the BigQuery query engine instead of your local environment.

Let's compute the mean of the `body_mass_g` series:

In [ ]:
average_body_mass = df["body_mass_g"].mean()
print(f"average_body_mass: {average_body_mass}")

Calculate the mean `body_mass_g` by `species` using the `groupby` operation:

In [ ]:
df[["species", "body_mass_g"]].groupby(by=df["species"]).mean(numeric_only=True)

You can confirm that the calculations were run in BigQuery by clicking "Open job" from the previous cells' output. This takes you to the BigQuery console to view the SQL statement and job details.

## Visualize data

### First party visualizations

BigQuery DataFrames provides a number of visualizations via the `plot` method and [accessor](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.operations.plotting.PlotAccessor) on the DataFrame and Series objects.

In [ ]:
df.plot(title="Numeric features")

In [ ]:
means = df.groupby("species").mean(numeric_only=True)
means.plot.bar()

### Integration with open source visualizations

BigQuery Dataframes is also compatible with several open source visualization packages, such as `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

# plotting a histogram
species_counts = df["species"].value_counts()
plt.pie(species_counts, labels=species_counts.index, autopct='%1.1f%%')
plt.show()

## Pandas interoperability

BigQuery DataFrames can be converted from and to Pandas DataFrame with `to_pandas` and `read_pandas` respectively.
This could be handy to take advantage of the capabilities of the two systems.

> Note: `to_pandas` converts the BigQuery DataFrame to Pandas DataFrame by bringing all the data in memory, which would be an issue
for large data, as your machine may not have enough memory to accommodate that.

In [ ]:
def print_type(df):
    print(f"\nWe have a dataframe of {type(df)}\n")

# The original bigframes dataframe
cur_df = df
print_type(cur_df)

# Convert to pandas dataframe
cur_df = cur_df.to_pandas()
print_type(cur_df)

# Convert back to bigframes dataframe
cur_df = bpd.read_pandas(cur_df)
print_type(cur_df)

## Machine Learning with BigQuery DataFrames

### Clean and prepare data

We're are going to start with supervised learning, where a Linear Regression model will learn to predict the body mass (output variable `y`) using input features such as flipper length, sex, species, and more (features `X`).

In [ ]:
# Drop any rows that has missing (NA) values
df = df.dropna()

# Isolate input features and output variable into DataFrames
X = df[['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'sex', 'species']]
y = df[['body_mass_g']]

# Print the shapes of features and label
print(f"""
    X shape: {X.shape}
    y shape: {y.shape}
""")

Part of preparing data for a machine learning task is splitting it into subsets for training and testing to ensure that the solution is not overfitting. By default, BQML will automatically manage splitting the data for you. However, BQML also supports manually splitting out your training data.

Performing a manual data split can be done with `bigframes.ml.model_selection.train_test_split` like so:

In [ ]:
from bigframes.ml.model_selection import train_test_split

# This will split X and y into test and training sets, with 20% of the rows in the test set,
# and the rest in the training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Show the shape of the data after the split
print(f"""
    X_train shape: {X_train.shape}
    X_test shape: {X_test.shape}
    y_train shape: {y_train.shape}
    y_test shape: {y_test.shape}
""")

### Define pipeline

This step is subjective to the problem. Although a model can be directly trained on the original data, it is often useful to apply some preprocessing to the original data.
In this example we want to apply a [`ColumnTransformer`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.compose.ColumnTransformer) in which we apply [`OneHotEncoder`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.preprocessing.OneHotEncoder) to the category features and [`StandardScaler`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.preprocessing.StandardScaler) to the numeric features.

In [ ]:
from bigframes.ml.linear_model import LinearRegression
from bigframes.ml.pipeline import Pipeline
from bigframes.ml.compose import ColumnTransformer
from bigframes.ml.preprocessing import StandardScaler, OneHotEncoder

preprocessing = ColumnTransformer([
    ("onehot", OneHotEncoder(), ["island", "species", "sex"]),
    ("scaler", StandardScaler(), ["culmen_depth_mm", "culmen_length_mm", "flipper_length_mm"]),
])

model = LinearRegression(fit_intercept=False)

pipeline = Pipeline([
    ('preproc', preprocessing),
    ('linreg', model)
])

# View the pipeline
pipeline

### Train and Predict

Supervised learning is when we train a model on input-output pairs, and then ask it to predict the output for new inputs. An example of such a predictor is `bigframes.ml.linear_models.LinearRegression`.

In [ ]:
# Learn from the training data how to predict output y
pipeline.fit(X_train, y_train)

# Predict y for the test data
y_pred = pipeline.predict(X_test)

# View predictions preview
y_pred.peek()

### Evaluate results

Some models include a convenient `.score(X, y)` method for evaulation with a preset accuracy metric:

In [ ]:
pipeline.score(X_test, y_test)

For a more general approach, the library `bigframes.ml.metrics` is provided:

In [ ]:
from bigframes.ml.metrics import r2_score

r2_score(y_test, y_pred["predicted_body_mass_g"])

## Generative AI with BigQuery DataFrames

BigQuery DataFrames integration with the Large Language Models (LLM) supported by BigQuery ML. Check out the [`bigframes.ml.llm`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.llm) module for all the available models.

### Create prompts

A "prompt" text column can be initialized either directly or via the pandas APIs. For simplicity let's use a direct initialization here.

In [ ]:
df = bpd.DataFrame(
        {
            "prompt": ["What is BigQuery?", "What is BQML?", "What is BigQuery DataFrames?"],
        })
df

### Generate responses

Here we will use the [`GeminiTextGenerator`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.llm.GeminiTextGenerator) LLM to answer the questions. Read the API documentation for all the model versions supported via the `model_name` param.

In [ ]:
from bigframes.ml.llm import GeminiTextGenerator

model = GeminiTextGenerator()

pred = model.predict(df)
pred

Let's print the full text response for the question "What is BigQuery DataFrames?".

In [ ]:
print(pred.loc[2]["ml_generate_text_llm_result"])

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

To remove any temporary cloud artifacts (inclusing BQ tables) created in the current BigQuery DataFrames session, simply call `close_session`.

In [ ]:
# Delete the temporary cloud artifacts created during the bigframes session 
bpd.close_session()

## Summary and next steps

1. You created BigQuery DataFrames objects, and inspected and manipulated data with pandas APIs at BigQuery scale and speed.

1. You also created ML model from a DataFrame and used them to run predictions on another DataFrame.

1. You got access to Google's state-of-the-art Gemini LLM through simple pythonic API.

Learn more about BigQuery DataFrames in the documentation [BigQuery DataFrames](https://cloud.google.com/bigquery/docs/bigquery-dataframes-introduction) and its [API reference](https://cloud.google.com/python/docs/reference/bigframes/latest).

Also, find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks), including the [pypi.ipynb](https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/dataframes/pypi.ipynb) that processes 400+ TB data at the cost and efficiency close to direct SQL by taking advantage of the [partial ordering](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes._config.bigquery_options.BigQueryOptions#bigframes__config_bigquery_options_BigQueryOptions_ordering_mode) mode.